# 3.Statistical analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
%matplotlib inline

In [2]:
train = pd.read_csv('../data/Preprocessed/train.csv')
test = pd.read_csv('../data/Preprocessed/test.csv')

In [5]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Cabin_First',
       'FamilyCount', 'title'],
      dtype='object')

In [6]:
# we will select just the columns that we want to use in our analysis
trainML = train[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Embarked', 'FamilyCount',
       'Cabin_First', 'title']]
trainML = trainML.dropna()

In [7]:
trainML.isnull().sum()


Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
FamilyCount    0
Cabin_First    0
title          0
dtype: int64

## 3.1 Starting regressions


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

### 3.1.1 Logistic Regression: Survival on age

In [ ]:
X_Age = trainML[['Age']].values
y = trainML['Survived'].values
# Use the fit method to train
lr.fit(X_Age,y)
# Make a prediction
y_predict = lr.predict(X_Age)
(y == y_predict).mean()

### 3.1.2 Logistic regression: Survival on sex

In [ ]:
X_sex = pd.get_dummies(trainML['Sex']).values
y = trainML['Survived'].values
# Use the fit method to train
lr.fit(X_sex, y)
# Make a prediction
y_predict = lr.predict(X_sex)
(y == y_predict).mean()

In [ ]:
### 3.1.3 random Random forest (on numerical variables)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X=trainML[['Age', 'SibSp', 'Parch',
       'Fare', 'FamilyCount']].values # Taking all the numerical values
y = trainML['Survived'].values
RF = RandomForestClassifier()
RF.fit(X, y)
# Make a prediction
y_predict = RF.predict(X)
y_predict[:10]
(y == y_predict).mean()
